# Notebook para classificação de de números da base MNIST
## Alexandre Suaide


## Primeiramente vamos importar as bibliotecas relevantes para esse exercício

In [ ]:
import numpy as np
import pickle
import gzip
import NN
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay

## Vamos definir uma função para ver um determinado dado na tabela e seu label

In [ ]:
def view(data, index):
    values = data[0][index]
    result = data[1][index]
    z = np.reshape(values,(28,28))
    plt.imshow(z)
    plt.gray()
    plt.show()
    print(f'O rótulo para esse dado é ==> {result}')

## Agora vamos determinar umas funções para organizar os dados em um formato que possamos usar na nossa rede neural

In [ ]:
def data_format(data, vectorize = False):
    inputs = [x.reshape(784,1) for x in data[0]]
    results = data[1]
    return inputs, results

## Lê o dataset

In [ ]:
file = gzip.open('data-mnist-numeros.gz', 'rb')
training, validation, test_data = pickle.load(file, encoding='latin1')
file.close()

## Formata os datasets e cria um dataset de treino mais curto

In [ ]:
x, y = data_format(training, True)
vx, vy = data_format(validation)
tx, ty = data_format(test_data)
x_short = x[:100]
y_short = y[:100]

## Vamos criar 2 redes neurais. Duas iguais com apenas uma camada escondida. Uma delas será treinada com todo dataset de treino e a outra, com o dataset menor. 

In [ ]:
nn1 = NN.neuralnet([784,20,10])
nn2 = NN.neuralnet([784,20,10])

## Treinando as redes

In [ ]:
nn1.fit(x, y, 10, 100, 3.0, tx, ty)

In [ ]:
nn2.fit(x_short, y_short, 100, 10, 3.0, tx, ty)

## Vendo algumas métricas de como o treinamento progrediu para cada rede com base no nosso critério de escolha de decisão

In [ ]:
fig = nn1.plot()

In [ ]:
fig = nn2.plot()

## Fazendo a rede tomar uma decisão com base em algum evento de validação

In [ ]:
index = 5769
view(test_data,index)

In [ ]:
nn1.output(tx[index])

In [ ]:
plt.plot(nn1.output(tx[index]))

In [ ]:
plt.plot(nn2.output(tx[index]))

## Aqui é para mostrar como a rede performa para ruído.
### Ela não foi treinada paa ruído. Não sabe nada sobre isso. Vai tomar uma decisão baseada apenas nos dados que ele foi treinada e no que aprendeu com eles. Você pode estar se perguntando, então, o que ela aprendeu? Aprendeu a reconhecer padrões ou só encontrou um mapa de transferência entre input e output?

In [ ]:
r = np.random.rand(784,1)
z = np.reshape(r,(28,28))
plt.imshow(z)
plt.gray()
plt.show()

## Saída da rede NN1, treinada com todo dataset de treino, para esse ruído. Ela pensa que é um número!

In [ ]:
plt.plot(nn1.output(r))

# Agora vamos fazer isso no KERAS

## Separando os datasets em nomes mais familiares e fazendo o output (Y) ser um vetor de 10 posições

In [ ]:
X_train = training[0]
Y_train = to_categorical(training[1])
X_test = test_data[0]
Y_test = to_categorical(test_data[1])
X_val = validation[0]
Y_val = to_categorical(validation[1])

## Construção do modelo igual à nossa rede anterior

In [ ]:
model = keras.Sequential([
    layers.Input(shape=(len(X_train[0]),)),
    layers.Dense(20, activation='sigmoid'),
    layers.Dense(10, activation='softmax') # essa é a unica diferença
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

## Treinando

In [ ]:
history = model.fit(
    X_train, Y_train,
    epochs=20,
    batch_size=128,
    validation_data=(X_val, Y_val),
    verbose=1
)

## faz figura da perda e da acurácia para cada época


In [ ]:
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.plot(history.history['loss'], label='treino')
plt.plot(history.history['val_loss'], label='validação')
plt.title('Perda')
plt.legend()

plt.subplot(1,2,2)
plt.plot(history.history['accuracy'], label='treino')
plt.plot(history.history['val_accuracy'], label='validação')
plt.title('Acurácia')
plt.legend()
plt.show()

## Avaliar o dataset de teste e obter a matriz de confusão

In [ ]:
test_loss, test_acc = model.evaluate(X_test, Y_test, verbose=0)
print(f"Test accuracy: {test_acc:.4f}")

y_pred_probs = model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)   # classe com maior probabilidade

Y_test_int = np.argmax(Y_test, axis=1)
cm = confusion_matrix(Y_test_int, y_pred)

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=np.arange(10))
disp.plot(cmap=plt.cm.Blues)
plt.title("Matriz de confusão - MNIST")
plt.show()

## Fazer uma previsão

In [ ]:
index = 1023
img = X_test[index]  # shape (784,)
img_input = np.expand_dims(img, axis=0)  # vira (1, 784)
pred = model.predict(img_input)
predicted_class = np.argmax(pred)
plt.imshow(img.reshape(28, 28), cmap='gray')
plt.title(f"Previsto: {predicted_class}, Verdadeiro: {np.argmax(Y_test[index])}")
plt.axis('off')
plt.show()

In [ ]:
plt.plot(pred[0])